# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [73]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [74]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [75]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [161]:
total_qty = data[["CustomerID","ProductName"]].groupby(["CustomerID","ProductName"]).sum()

In [162]:
total_qty.head()

Empty DataFrame
Columns: []
Index: [(33, Apricots - Dried), (33, Assorted Desserts), (33, Bandage - Flexible Neon), (33, Bar Mix - Pina Colada, 355 Ml), (33, Beans - Kidney, Canned)]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [222]:
total_matrix = data.pivot_table(values = 'Quantity', columns = "CustomerID", index = "ProductName", fill_value = 0)


In [223]:
total_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [315]:
def distancia(summary, metric='euclidean'):
    pairwise = pd.DataFrame(
        1/(1+squareform(pdist(summary, metric))),
        columns = summary.index,
        index = summary.index
    )
    return pairwise

In [316]:
euclidean_dist = distancia(total_matrix.T)

In [317]:
euclidean_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [328]:
def similar_user (user_id, n_top):
     return list(euclidean_dist[user_id].sort_values(ascending=False)[1:].head(n_top).index)
similar_to_33 = similar_user(33, 5)

In [329]:
similar_to_33

[3909, 264, 3531, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [331]:
records = data.loc[data['CustomerID'].isin(similar_user(33, 5))].sort_values("CustomerID")
records

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
32311,264,Virgil,Small,1609342,240,Wine - Chablis 2003 Champs,1
34764,264,Virgil,Small,4356324,289,Sword Pick Asst,1
33544,264,Virgil,Small,2585657,361,Coffee Decaf Colombian,1
32037,264,Virgil,Small,6577594,297,Hinge W Undercut,1
29028,264,Virgil,Small,5538534,151,"Mushrooms - Black, Dried",1
...,...,...,...,...,...,...,...
42462,3909,Keri,Stafford,2586356,451,Soup - Campbells Tomato Ravioli,1
42164,3909,Keri,Stafford,4953624,216,Scallops - 10/20,1
42007,3909,Keri,Stafford,5138975,202,Bay Leaf,1
3242,3909,Keri,Stafford,1925566,194,Lettuce - Frisee,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [330]:
products = records.groupby('ProductName', as_index=False).agg({'Quantity':'sum'})
products = products[['ProductName', 'Quantity']].sort_values(by='Quantity', ascending=False)
products.head()

,ProductName,Quantity
6,Bagel - Plain,100
163,"Pasta - Penne, Rigate, Dry",99
12,Bandage - Flexible Neon,98
18,Beef - Inside Round,75
52,"Cheese - Brie,danish",75


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [360]:
merged = pd.merge(products, records, on='ProductName')

,ProductName,Quantity_x,CustomerID,FirstName,LastName,SalesID,ProductID,Quantity_y
0,Bagel - Plain,100,264,Virgil,Small,6728381,262,1
1,"Pasta - Penne, Rigate, Dry",99,3305,Bernard,O'Neill,5796529,79,1
2,Bandage - Flexible Neon,98,264,Virgil,Small,2009347,273,1
3,Bandage - Flexible Neon,98,3305,Bernard,O'Neill,1274721,273,1
4,Beef - Inside Round,75,3305,Bernard,O'Neill,396005,248,1
...,...,...,...,...,...,...,...,...
196,Langers - Ruby Red Grapfruit,24,264,Virgil,Small,1307257,150,1
197,Tea - Herbal Sweet Dreams,24,3531,Ivan,Buck,1659182,141,1
198,Kellogs Special K Cereal,24,3305,Bernard,O'Neill,901895,16,1
199,Kellogs Special K Cereal,24,3531,Ivan,Buck,4868523,16,1


In [362]:
not_purchase = merged[merged["CustomerID"]!=33]

In [365]:
list(pd.unique(not_purchase['ProductName'].values))[:5]

['Bagel - Plain',
 'Pasta - Penne, Rigate, Dry',
 'Bandage - Flexible Neon',
 'Beef - Inside Round',
 'Wine - Blue Nun Qualitatswein']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.